In [21]:
import pandas as pd
from dotenv import load_dotenv
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing	import OneHotEncoder
from datetime import datetime
import pickle
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.tree import DecisionTreeClassifier

load_dotenv()
os.chdir(os.getenv("ROOT_DIR"))

from src import utils

In [22]:
# --- PARAMS --- #
prep_id = 'prep_01'
model_id = 'model_01'
data_to_predict_path = os.getenv("DF_NEW_DATA_PATH")

print(data_to_predict_path)

data/raw/test.csv


In [23]:
# --- LOAD DATA --- #
features = ['Pclass', 'Sex']
target = ['Survived']

df_to_predict = pd.read_csv(data_to_predict_path)
x_predict = df_to_predict[features]
x_predict.head(2)

,Pclass,Sex
0,3,male
1,3,female


In [24]:
# --- PREPROCESSING --- #

# sex
with open(os.path.join(os.getenv("ARTIFACTS_PATH"), prep_id, 'encoder_sex_model.pkl'), 'rb') as f:
          encoder_sex_model = pickle.load(f)

x_predict[encoder_sex_model.get_feature_names_out()] = encoder_sex_model.transform(x_predict[['Sex']])
x_predict.drop('Sex', axis=1, inplace=True)

/tmp/ipykernel_1153485/3502322382.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_predict[encoder_sex_model.get_feature_names_out()] = encoder_sex_model.transform(x_predict[['Sex']])
/tmp/ipykernel_1153485/3502322382.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_predict[encoder_sex_model.get_feature_names_out()] = encoder_sex_model.transform(x_predict[['Sex']])
/tmp/ipykernel_1153485/3502322382.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Se

In [29]:
# --- PREDICT --- #
model_path = os.path.join(os.getenv("ARTIFACTS_PATH"), model_id, 'model', 'model.pkl')
with open(model_path, 'rb') as f:
    model = pickle.load(f)

y_predict = model.predict(x_predict)

res = df_to_predict[['PassengerId']]
res['Survived'] = y_predict

pred_path = os.path.join(os.getenv("ARTIFACTS_PATH"), utils.get_nb_name(), 'submission.csv')
os.makedirs(os.path.dirname(pred_path), exist_ok=True)
res.to_csv(pred_path, index=False)
print(f'Predictions saved at {pred_path}')

res.head()

Predictions saved at artifacts/model_01b_predict/submission.csv


/tmp/ipykernel_1153485/1621607376.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['Survived'] = y_predict


,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [28]:
# --- SUBMIT --- #
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [20]:
#! kaggle competitions submit -c titanic -f {pred_path} -m "submission model01b_predict"

100%|██████████████████████████████████████| 2.77k/2.77k [00:00<00:00, 3.09kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster